In [33]:
import pandas as pd
import geopandas as gpd
import unidecode
import os

path_regioes = "divisoes_regionais.csv" 
path_limites_mun = "Shapefiles_auxiliares\limite_municipal\LimiteMunicipal.shp"
path_PIB = "Processamento_base_PIB_MUN\PIB_processado.csv"
path_ipdm = "Processamento_base_IPDM\ipdm_processada.csv"
path_est_pop = "Processamento_base_EST_POP\EST_POP_processada.csv"
path_regiao_imediata = "Shapefiles_auxiliares\regiao_geografica_imediata\Regiao Geografica Imediata.shp"

### Processamento Individual de cada Base

In [34]:
divisao_regional = pd.read_csv(path_regioes)
divisao_regional = divisao_regional[divisao_regional['Nome_UF'] == 'São Paulo']

divisao_regional.drop(columns=['UF', 'Nome_UF', 'Mesorregião Geográfica',
                      'Nome_Mesorregião', 'Microrregião Geográfica',
                      "Nome_Microrregião", "Município"], inplace=True)

divisao_regional.rename(columns={'Código Município Completo': 'cod_ibge',
                        'Nome_Município': 'Municipio'}, inplace=True)

In [35]:
limites_regionais = gpd.read_file(path_limites_mun)
limites_regionais.drop(columns=['GID_RA','GID_RG','GID_RM', 'GID_RM', 'RM', 'AU', 'GID_AU', 'RA', 'RG'], inplace=True)
limites_regionais.rename(columns={'Cod_ibge': 'cod_ibge',
                                  'Municipios': 'municipio'}, inplace=True)

In [36]:
PIB = pd.read_csv(path_PIB, index_col=0)
# O PIB não possui registros para o ano de 2022
PIB = PIB[PIB['Ano'].isin([2014, 2016, 2018, 2020])]

In [37]:
EST_POP = pd.read_csv(path_est_pop, index_col=0)
EST_POP = EST_POP[EST_POP['Ano'].isin([2014, 2016, 2018, 2020])]
cols = ['razao_sexo', 'id_media', 'dens_demog']
EST_POP[cols] = EST_POP[cols].replace({',': '.'}, regex=True).apply(pd.to_numeric)

In [38]:
ipdm = pd.read_csv(path_ipdm, index_col=0)
ipdm.drop(columns=['Produto Interno Bruto per capita'], inplace=True)

In [39]:
ipdm = ipdm.merge(divisao_regional, on=['cod_ibge', 'Municipio'], how='inner')

### Junção das Bases de acordo com o ano

In [40]:
def juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, ano):
    
    caminho_indices_criminais = f"indices_criminais\{ano}\indices_criminais{ano}.csv"
    indices_criminais = pd.read_csv(caminho_indices_criminais)
        
    ipdm_ano = ipdm[ipdm['Ano'] == ano].copy()
    EST_pop_ano = EST_POP[EST_POP['Ano'] == ano].copy()
    PIB_ano = PIB[PIB['Ano'] == ano].copy()
    
    #Padronização dos municipios
    ipdm_ano['Municipio'] = ipdm_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    EST_pop_ano['Municipio'] = EST_pop_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    PIB_ano['Municipio'] = PIB_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    limites_regionais['Municipio'] = limites_regionais['Municipio'].apply(lambda x: unidecode.unidecode(x))
    indices_criminais['Municipio'] = indices_criminais['Municipio'].apply(lambda x: unidecode.unidecode(x))
    
    df_merge1 = pd.merge(ipdm_ano, EST_pop_ano, on=['Ano', 'cod_ibge', 'Municipio'], how='inner')
    df_final = pd.merge(df_merge1, PIB_ano, on=['Ano', 'Municipio'], how='inner')
    df_final = limites_regionais.merge(df_final, on=['cod_ibge', 'Municipio'], how='inner')
    df_final = df_final.merge(indices_criminais, on=['cod_ibge', 'Municipio'], how='inner')
    
    df_final.drop(columns={'Ano_y'}, inplace=True)
    df_final.rename(columns={'Ano_x': 'Ano'}, inplace=True)
    
    return df_final
    
    
df_2014 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB,  2014)
df_2016 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB,  2016)
df_2018 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB,  2018)
df_2020 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB,  2020)

In [42]:
def renomear_colunas_por_ano(df, ano):
    """
    Renomeia colunas de um DataFrame para siglas curtas, adicionando o ano ao final.
    """
    # Mapeamento base de colunas para siglas
    base_colunas_renomeadas = {
        'Consumo anual de energia elétrica residencial': 'CAER',
        'Consumo anual de energia elétrica comercial': 'CAEC',
        'Rendimento do trabalho formal': 'RTF',
        'Indicador Riqueza': 'IndRiq',
        'Taxas de mortalidade infantil': 'TMI',
        'Taxas de mortalidade perinatal': 'TMP',
        'Taxas de mortalidade 15 a 39 anos': 'TM1539',
        'Taxas de mortalidade 60 a 69 anos': 'TM6069',
        'Indicador Longevidade': 'IndLong',
        'Taxa de atendimento escolar na faixa etária de 0 a 3 anos': 'TAE03',
        'Proporção média de alunos do 5º ano com proficiência em Língua Portuguesa e Matemática': 'Prof5ano',
        'Proporção média de alunos do 9º ano com proficiência em Língua Portuguesa e Matemática': 'Prof9ano',
        'Taxas de distorção idade-série no Ensino Médio': 'TDISM',
        'Indicador Escolaridade': 'IndEsc',
        'Região Geográfica Intermediária': 'RGInter',
        'Nome Região Geográfica Intermediária': 'NRGInter',
        'Região Geográfica Imediata': 'RGIme',
        'Nome Região Geográfica Imediata': 'NRGIme',
        'Agropecuária': 'Agrop',
        'Administração pública': 'AdmPub',
        'Serviços (exceto administração pública)': 'ServExc',
        'Valor adicionado': 'ValAdic',
        'Impostos líquido de subsídios': 'ImpLiq',
        'PIB per capita': 'PIBpc'
    }

    # Adicionar o ano às siglas para as colunas específicas de cada ano
    colunas_ano = {
        'Furto de veículo': 'FV',
        'Furto outros': 'FO',
        'Homicídio culposo outros': 'HC',
        'Homicídio culposo por acidente de trânsito': 'HCA',
        'Latrocínio': 'Lat',
        'Lesão corporal culposa outras': 'LCC',
        'Lesão corporal culposa por acidente de trânsito': 'LCCA',
        'Lesão corporal dolosa': 'LCD',
        'Vítimas de homicídio': 'VH',
        'Tentativa de homicídio': 'TH',
        'Total de estupro': 'TE',
        'Total de Roubo': 'TR',
        'Indice Criminalidade': 'IC',
        'Classificação': 'CL'
    }

    # Atualizar nomes com ano para colunas específicas
    colunas_renomeadas = {**base_colunas_renomeadas}
    for coluna, sigla in colunas_ano.items():
        colunas_renomeadas[f"{coluna} {ano}"] = f"{sigla}{ano}"

    # Renomear as colunas no DataFrame
    df.rename(columns=colunas_renomeadas, inplace=True)
    return df

# Aplicar a função para os DataFrames de diferentes anos
df_2014 = renomear_colunas_por_ano(df_2014, 2014)
df_2016 = renomear_colunas_por_ano(df_2016, 2016)
df_2018 = renomear_colunas_por_ano(df_2018, 2018)
df_2020 = renomear_colunas_por_ano(df_2020, 2020)

In [43]:
pd.set_option('display.max_columns', None)
df_2014

,cod_ibge,Municipio,Area_Km2,geometry,Ano,IPDM,CAER,CAEC,RTF,IndRiq,TMI,TMP,TM1539,TM6069,IndLong,TAE03,Prof5ano,Prof9ano,TDISM,IndEsc,RGInter,NRGInter,RGIme,NRGIme,populacao,homens,mulheres,razao_sexo,id_media,dens_demog,Agrop,Indústria,AdmPub,ServExc,ValAdic,ImpLiq,PIB,PIBpc,FV2014,FO2014,HC2014,HCA2014,Lat2014,LCC2014,LCCA2014,LCD2014,VH2014,TH2014,TE2014,TR2014,IC2014,CL2014
0,3509908,Cananeia,1237.357,"POLYGON ((-47.83033 -24.75638, -47.83013 -24.7...",2014,0.480,2.562923,8.001560,644.220358,0.339,7.604563,9.451796,1.321371,20.474138,0.725,21.224490,42.018494,17.283870,13.888889,0.375,3502,Sorocaba,350005,Registro,12311.0,6122.0,6189.0,101.1,34.3,9.9,26175.315945,1.955748e+05,48919.100281,134460.454610,4.051296e+05,7911.439205,4.130411e+05,33814.251676,1,159,1,2,0,0,13,93,1,1,12,9,0.367274,Baixo
1,3520426,Ilha Comprida,196.567,"POLYGON ((-47.4366 -24.67434, -47.43638 -24.67...",2014,0.534,3.706455,8.709898,815.521197,0.501,10.856454,11.990408,2.117074,19.886364,0.650,42.352941,50.351199,23.858752,20.481928,0.450,3502,Sorocaba,350005,Registro,10507.0,5212.0,5295.0,101.6,37.9,53.5,2507.249580,2.721854e+06,74064.530665,992619.188352,3.791045e+06,27361.710783,3.818407e+06,401303.948908,12,636,0,4,0,0,68,121,1,8,10,50,1.000000,Muito Alto
2,3505401,Barra do Turvo,1007.670,"POLYGON ((-48.44457 -24.65134, -48.44418 -24.6...",2014,0.335,1.498860,3.513185,572.414524,0.201,16.949153,18.181818,2.331002,12.987013,0.595,2.286902,18.974225,6.391028,13.436693,0.210,3502,Sorocaba,350005,Registro,7492.0,3895.0,3597.0,92.3,33.9,7.4,4637.263666,3.425711e+03,29195.312005,27979.235527,6.523752e+04,5244.742273,7.048226e+04,9184.553606,0,74,0,9,0,0,16,34,4,3,1,50,0.273911,Baixo
3,3509254,Cajati,454.436,"POLYGON ((-48.1556 -24.64078, -48.15505 -24.64...",2014,0.443,1.778662,6.070423,1237.874551,0.337,14.363885,18.957346,1.601739,15.692948,0.626,16.376496,38.551838,16.047736,8.808664,0.367,3502,Sorocaba,350005,Registro,28622.0,14438.0,14184.0,98.2,32.2,63.0,135215.223225,4.323512e+05,106129.593657,308653.931094,9.823500e+05,112565.534714,1.094916e+06,38446.416903,29,300,0,13,0,0,65,127,3,5,20,72,0.252907,Baixo
4,3524600,Jacupiranga,704.189,"POLYGON ((-48.00104 -24.5716, -48.0004 -24.572...",2014,0.479,1.803224,5.102513,900.044501,0.286,12.482663,12.413793,1.664708,18.171956,0.670,25.473684,56.376474,22.153228,6.809079,0.481,3502,Sorocaba,350005,Registro,16875.0,8528.0,8347.0,97.9,33.7,24.0,73600.998952,2.353773e+04,58505.345273,168053.876483,3.236980e+05,21424.341765,3.451223e+05,20106.163524,11,128,0,6,0,2,52,101,1,1,7,31,0.273059,Baixo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,3530003,Mira Estrela,216.825,"POLYGON ((-50.10224 -19.87441, -50.09732 -19.8...",2014,0.544,1.742547,5.400891,683.838622,0.261,10.526316,10.471204,1.958864,14.023732,0.703,79.816514,68.396770,29.834062,4.854369,0.669,3507,São José do Rio Preto,350029,Fernandópolis,2919.0,1452.0,1467.0,101.0,38.8,13.5,8702.953926,2.310827e+03,15342.040496,15984.764324,4.234059e+04,1334.146551,4.367473e+04,15222.980780,0,14,0,0,0,0,3,9,0,0,1,0,0.117943,Muito Baixo
636,3520707,Indiapora,279.606,"POLYGON ((-50.20045 -19.86626, -50.19701 -19.8...",2014,0.549,1.910248,6.378619,826.516748,0.286,13.468013,16.722408,0.722022,14.354067,0.699,51.308901,79.014845,40.625000,15.972222,0.661,3507,São José do Rio Preto,350029,Fernandópolis,3974.0,1949.0,2025.0,103.9,39.6,14.2,14309.599014,4.514851e+03,16792.143906,25911.709110,6.152830e+04,1494.346246,6.302265e+04,16259.713359,1,12,0,3,0,1,17,25,0,0,2,1,0.255380,Baixo
637,3529658,Mesopolis,148.636,"POLYGON ((-50.61347 -19.86537, -50.61346 -19.8...",2014,0.524,1.701754,7.133663,504.006583,0.269,17.964072,11.904762,1.436782,17.647059,0.656,55.670103,58.000000,47.368421,12.328767,0.646,3507,São José do Rio Preto,350028,Jales,1901.0,966.0,935.0,96.8,36.6,12.8,19332.888660,2.110240e+03,110

In [44]:
def exportar_df(gdf, ano, caminho_destino='Base_final'):
    diretorio = os.path.join(caminho_destino, str(ano))
    os.makedirs(diretorio, exist_ok=True)
    
    arquivo = os.path.join(diretorio, f"{ano}.shp")
    gdf.to_file(arquivo, driver='ESRI Shapefile')
    print(f"Exportado: {arquivo}")    
    
exportar_df(df_2014, 2014)
exportar_df(df_2016, 2016)
exportar_df(df_2018, 2018)
exportar_df(df_2020, 2020)

c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 406109012.608356714 of field ServExc of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 510873162.175711572 of field ValAdic of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 111044209.473889217 of field ImpLiq of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 621917371.649600863 of field PIB of feature 90 not successful

Exportado: Base_final\2014\2014.shp


c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 463376974.425000012 of field ServExc of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 565960106.820000052 of field ValAdic of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 117106590.018999994 of field ImpLiq of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 683066696.838999987 of field PIB of feature 90 not successful

Exportado: Base_final\2016\2016.shp


c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 485395448.600000024 of field ServExc of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 587506566.893000007 of field ValAdic of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 127157037.150000006 of field ImpLiq of feature 90 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 714663604.042000055 of field PIB of feature 90 not successful

Exportado: Base_final\2018\2018.shp
Exportado: Base_final\2020\2020.shp


c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 520357968.893000007 of field ServExc of feature 91 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 624409861.152999997 of field ValAdic of feature 91 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 124349145.812000006 of field ImpLiq of feature 91 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\muril\Documents\2024\projeto_analise_criminal\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 748759006.965000033 of field PIB of feature 91 not successful